<a href="https://colab.research.google.com/github/lpoggetto/fiap_liveness/blob/main/trabalho_computer_vision_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atenção!

Recomendado utilizar instância com placa de vídeo!

Ambiente de execução -> Alterar o tipo de ambiente de execução -> T4 GPU

### Limpando a pasta do colab para iniciar um novo projeto

In [10]:
# limpando a pasta
!rm -rf /content/*

### Instalando os pacotes utilizados no programa

In [2]:
# instalando pacotes utilizados
!pip uninstall -y dlib


!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
!pip install face_detection
!pip install mediapipe
# !pip install dlib

!apt-get update
!apt-get install -y cmake libopenblas-dev liblapack-dev libx11-dev libgtk-3-dev
!pip install dlib --no-binary :all:

print('pacotes instalados')

Found existing installation: dlib 19.24.6
Uninstalling dlib-19.24.6:
  Successfully uninstalled dlib-19.24.6
  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-ydwjsbtp
  Running command git clone --filter=blob:none --quiet https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-ydwjsbtp
  Resolved https://github.com/hukkelas/DSFD-Pytorch-Inference.git to commit dde9c7dd9cdc9254c2ca345222c86a8ecfa17f5b
  Preparing metadata (setup.py) ... done
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

### Importando os pacotes utilizados

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import bz2
import os
import face_detection
import mediapipe as mp
from scipy.spatial import distance as dist
from PIL import Image
import warnings
from google.colab import output
warnings.filterwarnings('ignore')

#Exibição na mesma tela do Jupyter
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})

# pacotes utilizados para tirar foto/videos
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode
import threading

import dlib

In [9]:
print("dlib version:", dlib.__version__)
print("CUDA available:", dlib.DLIB_USE_CUDA)

dlib version: 19.24.6
CUDA available: False


### Criacao de pasta pasta para armazenamento dos videos

In [4]:
# criando diretorios para armazenar as informacoes necessarias
os.makedirs('videos', exist_ok=True) # validacao de video
os.makedirs('foto', exist_ok=True) # validacao de video

### Importando os modelos utilizados no reconhecimento facial

In [5]:
# URL e nome dos arquivos
landmark_url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
landmark_file_name = "shape_predictor_68_face_landmarks.dat.bz2"
landmark_dat_file = "shape_predictor_68_face_landmarks.dat"

recognition_url = "http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_file_name = "dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_dat_file = "dlib_face_recognition_resnet_model_v1.dat"

# Criando diretório para salvar os modelos
os.makedirs('dlib_models', exist_ok=True)

# Apontando para o diretório
os.chdir('dlib_models')

# Função para baixar e extrair arquivos
def download_and_extract(url, compressed_file_name, extracted_file_name):
    # Baixando o arquivo se já não estiver baixado
    if not os.path.exists(compressed_file_name):
        os.system(f"wget {url}")

    # Extraindo o arquivo .dat se já não estiver extraído
    if not os.path.exists(extracted_file_name):
        with bz2.BZ2File(compressed_file_name, "rb") as f_in, open(extracted_file_name, "wb") as f_out:
            f_out.write(f_in.read())

        # Apagando o arquivo compactado
        os.remove(compressed_file_name)

# Baixando e extraindo o arquivo de landmarks
download_and_extract(landmark_url, landmark_file_name, landmark_dat_file)

# Baixando e extraindo o arquivo de reconhecimento facial
download_and_extract(recognition_url, recognition_file_name, recognition_dat_file)

# Voltando para o diretório original
os.chdir('..')

print('Modelos baixados e extraídos com sucesso!')

Modelos baixados e extraídos com sucesso!


### Capturando foto "original" para validar faces

In [6]:
def capturar_foto_colab():
    """Captura uma imagem no Google Colab usando a webcam."""
    display(Javascript('''
        async function captureImage() {
            const div = document.createElement('div');
            const captureButton = document.createElement('button');
            captureButton.textContent = 'Capture';
            div.appendChild(captureButton);
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => captureButton.onclick = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg').split(',')[1];
        }
    '''))
    image_data = output.eval_js('captureImage()')
    with open('/content/foto/foto_base.jpg', 'wb') as f:
        f.write(b64decode(image_data))
    print("Imagem capturada e salva como 'foto_base.jpg'.")

capturar_foto_colab()

<IPython.core.display.Javascript object>

Imagem capturada e salva como 'foto_base.jpg'.


### Capturando video para aplicar liveness

In [ ]:
# JavaScript para capturar video
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" download="recorded-video.webm"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                playback.src = URL.createObjectURL(blob);
                downloadLink.href = playback.src;
                downloadLink.textContent = 'Download Video';
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Display the HTML interface
display(HTML(html_code))

### Montando detector de faces

* Utilizando o algoritmo DSFDDetector para detectar uma face
* Caso seja identificado que o usuario piscou os olhos, a prova de vida sera considerada valida

In [7]:
def calculo_distancia_olhos(eye):
    # Calcula a distância euclidiana entre os pontos dos olhos
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # Calcula a razão de aspecto do olho
    ear = (A + B) / (2.0 * C)
    return ear

def detector_faces_video(video_path, output_path, threshhold_ear=0.3, ref_image_path='/content/foto/foto_base.jpg', treshhold_recognition = 0.6):
    # Criando o detector de faces
    detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=0.5,
        nms_iou_threshold=0.3
    )
    # utilizando algoritmo de 68 pontos faciais
    preditor_marcos_faciais = dlib.shape_predictor(
        '/content/dlib_models/shape_predictor_68_face_landmarks.dat'
        )

    # algoritmo de face recognition
    reconhecedor = dlib.face_recognition_model_v1(
        '/content/dlib_models/dlib_face_recognition_resnet_model_v1.dat'
        )

    # Carregando a imagem de referência
    ref_image = dlib.load_rgb_image(ref_image_path)
    ref_detections = detector.detect(ref_image)

    if len(ref_detections) == 0:
        raise ValueError('Nenhuma face detectada na imagem de referência.')

    x1, y1, x2, y2, score = map(int, ref_detections[0])
    ref_dlib_rect = dlib.rectangle(x1, y1, x2, y2)
    ref_landmarks = preditor_marcos_faciais(ref_image, ref_dlib_rect)
    ref_embedding = np.array(reconhecedor.compute_face_descriptor(ref_image, ref_landmarks))

    # abrindo o arquivo de video
    video = cv2.VideoCapture(video_path)

    # propriedades do video
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'VP80')  # WebM codec
    out = cv2.VideoWriter(output_path, fourcc, 15, (width, height))

    # iniciando variaveis para a validacao de prova vida
    blink_detected = False
    fl_liveness = 0

    # loop para avaliar o video
    while True:
        ret, frame = video.read()
        if not ret:
            break

        # convertendo para RGB
        rgb_frame = frame[:, :, ::-1]

        # detectando  face
        detections = detector.detect(rgb_frame)

        # a partir do momento em que e identificado que o usuario piscou os olhos
        # a variavel de liveness redefine para 1
        if blink_detected == True:
            fl_liveness = 1

        # desenhando o retangulo na face
        for det in detections:
            x1, y1, x2, y2, score = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])


            # utilizando dlib para detectar a face
            dlib_rect = dlib.rectangle(x1,y1,x2,y2)

            # marcos faciais
            landmarks = preditor_marcos_faciais(rgb_frame, dlib_rect)

            # coordenadas dos olhos
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # calculando a distancia para ambos os olhos
            olho_esq = calculo_distancia_olhos(left_eye)
            olho_dir = calculo_distancia_olhos(right_eye)
            ear = (olho_esq + olho_dir) / 2

            # validacao de olhos piscando
            blink_detected = ear < threshhold_ear

            # computando a face embedding
            face_embedding = np.array(reconhecedor.compute_face_descriptor(rgb_frame, landmarks))

            # Comparando com a face de referência
            similarity = 1 - dist.cosine(ref_embedding, face_embedding)

            # Reconhecimento facial
            if similarity > treshhold_recognition:
                recognition_text = "Reconhecido"
                color = (0, 255, 0)  # Verde
            else:
                recognition_text = "Nao Reconhecido"
                color = (0, 0, 255)  # Vermelho

            # Caixa ao redor da face
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, recognition_text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Checando se identifica o usuário piscando os olhos
            if fl_liveness == 1:
                # Caixa verde - Liveness OK
                cv2.putText(frame, 'Liveness OK', (x1, y2 + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # Caixa vermelha - Liveness não OK
                cv2.putText(frame, 'Liveness nao OK', (x1, y2 + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # retorno do video
        out.write(frame)

    # Release resources
    video.release()
    out.release()
    print(f"Video salvo em: {output_path}")

# variaveis para uso da funcao
input_video_path = '/content/videos/recorded-video.webm'
output_video_path = '/content/videos/recorded-video_processado.webm'
ref_image_path = '/content/foto/foto_base.jpg'

# chamada da funcao criada anteriormente
detector_faces_video(input_video_path, output_video_path, threshhold_ear=0.3, ref_image_path=ref_image_path, treshhold_recognition=0.6)

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-4ukagg0_/dlib_8e5c4a6b9fbc4884804eca21649b2ede/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version